In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
import re
from itertools import chain
import pickle
import joblib

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joao.victor.ribeiro\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joao.victor.ribeiro\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
artists = pd.read_csv('artists-data.csv')
songs = pd.read_csv('lyrics-data.csv')

In [3]:
genres_list = [str(s).split(';') for s in artists['Genres'].unique()]
res = list(chain(*genres_list))
res = [gen.strip() for gen in res]
genres = list(set(res))

In [4]:
for genre in genres:
    contains = [True if re.search(genre, str(art_gen)) else False for art_gen in artists['Genres']]
    artists[genre] = contains

In [5]:
all_songs = songs.merge(artists, how='outer', left_on='ALink', right_on='Link')

In [6]:
no_country = all_songs[all_songs['Country']==False].sample(n=2500, random_state=1)
yes_country = all_songs[all_songs['Country']==True].sample(n=2500, random_state=1)
no_rock = all_songs[all_songs['Rock']==False].sample(n=2500, random_state=1)
yes_rock = all_songs[all_songs['Rock']==True].sample(n=2500, random_state=1)
no_rap = all_songs[all_songs['Rap']==False].sample(n=2500, random_state=1)
yes_rap = all_songs[all_songs['Rap']==True].sample(n=2500, random_state=1)

In [7]:
all_songs = pd.concat([no_country, yes_country, no_rock, yes_rock, no_rap, yes_rap]).reset_index(drop=True)

In [8]:
lyrics = all_songs['Lyric'].astype(str)
low = [lyr.lower() for lyr in lyrics]
tokenized = [word_tokenize(lyr) for lyr in low]
stop_words = set(stopwords.words('english'))
stop_vec = [[w for w in tok if w not in stop_words] for tok in tokenized]
clean_vec = [[word for word in lyr if word.isalpha()] for lyr in stop_vec]

In [9]:
lyrics = [' '.join(lyr) for lyr in clean_vec]
vectorize = TfidfVectorizer(min_df=5, max_df=0.8)
vectors = vectorize.fit_transform(lyrics)
feature_names = vectorize.get_feature_names_out()
dense = vectors.todense()
dense_list = dense.tolist()
df = pd.DataFrame(dense_list, columns=feature_names)

In [10]:
df['Country'] = all_songs['Country'].copy()
df['Rock'] = all_songs['Rock'].copy()
df['Rap'] = all_songs['Rap'].copy()

In [11]:
genres = ['Country', 'Rock', 'Rap']
models = {}

In [12]:
for genre in genres:
    y = df[genre].copy()
    x = df.drop(genres, axis=1)  # Remove all genre columns
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    
    clf = MultinomialNB(alpha=0.1)
    clf.fit(x_train, y_train.astype(bool))
    models[genre] = clf
    
    y_pred = clf.predict(x_test)

    accuracy = metrics.accuracy_score(y_test.astype(bool), y_pred)
    precision = metrics.precision_score(y_test.astype(bool), y_pred)
    recall = metrics.recall_score(y_test.astype(bool), y_pred)
    confusion_matrix = metrics.confusion_matrix(y_test.astype(bool), y_pred)
    
    print(f"Results for {genre}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Confusion Matrix: \n{confusion_matrix}\n")

Results for Country:
Accuracy: 0.8253333333333334
Precision: 0.6262626262626263
Recall: 0.21602787456445993
Confusion Matrix: 
[[3528  111]
 [ 675  186]]

Results for Rock:
Accuracy: 0.7206666666666667
Precision: 0.5460434983803795
Recall: 0.8104395604395604
Confusion Matrix: 
[[2063  981]
 [ 276 1180]]

Results for Rap:
Accuracy: 0.9042222222222223
Precision: 0.7940761636107193
Recall: 0.6639150943396226
Confusion Matrix: 
[[3506  146]
 [ 285  563]]



In [13]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

In [14]:
file_path = 'lyric.txt'
text_content = read_txt_file(file_path)
df = pd.DataFrame({'lyrics': [text_content]})
df

,lyrics
0,I used to spend my nights out in a barroom\nLi...


In [15]:
csv_path = 'lyric.csv'

df.to_csv(csv_path, index=False)

In [16]:
tay = pd.read_csv('lyric.csv', encoding='latin1')
tay.columns
teste = tay['lyrics']
tokenized = [word_tokenize(lyr) for lyr in teste.astype(str)]
stop_vec = [[w for w in tok if w not in stop_words] for tok in tokenized]
clean_vec = [[word for word in lyr if word.isalpha()] for lyr in stop_vec]
wnet = nltk.WordNetLemmatizer()
lem = [[wnet.lemmatize(w) for w in lyr] for lyr in clean_vec]

lem

[['I',
  'used',
  'spend',
  'night',
  'barroom',
  'Liquor',
  'love',
  'known',
  'But',
  'rescued',
  'bottom',
  'And',
  'brought',
  'back',
  'far',
  'gone',
  'smooth',
  'Tennessee',
  'whiskey',
  'sweet',
  'strawberry',
  'wine',
  'warm',
  'glass',
  'brandy',
  'And',
  'honey',
  'I',
  'stay',
  'stoned',
  'love',
  'time',
  'looked',
  'love',
  'old',
  'place',
  'Found',
  'bottom',
  'bottle',
  'always',
  'dry',
  'But',
  'poured',
  'heart',
  'I',
  'waste',
  'nothing',
  'like',
  'love',
  'get',
  'high',
  'smooth',
  'Tennessee',
  'whiskey',
  'sweet',
  'strawberry',
  'wine',
  'warm',
  'glass',
  'brandy',
  'And',
  'honey',
  'I',
  'stay',
  'stoned',
  'love',
  'time',
  'smooth',
  'Tennessee',
  'whiskey',
  'sweet',
  'strawberry',
  'wine',
  'warm',
  'glass',
  'brandy',
  'And',
  'honey',
  'I',
  'stay',
  'stoned',
  'love',
  'time',
  'You',
  'smooth',
  'Tennessee',
  'whiskey',
  'Tennessee',
  'whiskey',
  'Tennessee',
 

In [17]:
lyrics_tay = [' '.join(lyr) for lyr in lem]
single_entry = vectorize.transform(lyrics_tay)
s_e = single_entry.todense().tolist()

lyrics_tay

['I used spend night barroom Liquor love known But rescued bottom And brought back far gone smooth Tennessee whiskey sweet strawberry wine warm glass brandy And honey I stay stoned love time looked love old place Found bottom bottle always dry But poured heart I waste nothing like love get high smooth Tennessee whiskey sweet strawberry wine warm glass brandy And honey I stay stoned love time smooth Tennessee whiskey sweet strawberry wine warm glass brandy And honey I stay stoned love time You smooth Tennessee whiskey Tennessee whiskey Tennessee whiskey You smooth Tennessee whiskey Tennessee whiskey Tennessee whiskey']

In [18]:
predictions = {genre: clf.predict(s_e) for genre in genres}

c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [21]:
probabilities = {genre: models[genre].predict_proba(s_e)[0][1] for genre in genres}

for genre, prob in probabilities.items():
    print(f"{genre}: {prob * 100:.2f}%")
    with open(f"{genre}_model.pkl", "wb") as f:
        pickle.dump(models[genre], f)

Country: 80.62%
Rock: 48.71%
Rap: 9.14%


c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [20]:
with open("vectorize.pkl", "wb") as f:
    pickle.dump(vectorize, f)